# 장기 메모리 (Long-term Memory)

LangChain 에이전트는 **LangGraph persistence** 기능을 사용하여 **장기 메모리(long-term memory)** 를 구현할 수 있습니다.
이 기능은 고급 주제에 속하며, 이를 사용하려면 **LangGraph의 동작 원리에 대한 이해**가 필요합니다.

LangGraph는 장기 메모리를 **JSON 문서 형식**으로 저장하며,
이를 **store(저장소)** 에 보관합니다.

각 메모리는 다음과 같이 구성됩니다:

* **namespace (네임스페이스)** → 폴더와 비슷한 개념으로, 데이터를 그룹화하는 단위입니다.
* **key (키)** → 파일 이름처럼, 해당 메모리를 구분하는 고유 식별자입니다.

보통 `namespace`에는 사용자 ID, 조직 ID 등의 정보를 포함하여
데이터를 논리적으로 분류하고 관리하기 쉽게 합니다.

이러한 구조를 통해 **계층적(hierarchical) 메모리 관리**가 가능하며,
**콘텐츠 필터(content filter)** 를 활용한 **교차-네임스페이스 검색**도 지원됩니다.


| 구분    | 단기 메모리 (short-term memory)                    | 장기 메모리 (long-term memory)          |
| ----- | --------------------------------------------- | ---------------------------------- |
| 저장 범위 | 현재 스레드(thread)나 세션 내 메시지                      | 여러 스레드, 여러 세션에 걸친 데이터              |
| 지속성   | 일시적 (대화 종료 시 사라짐)                             | 지속적 (DB나 파일에 저장)                   |
| 예시    | “방금 말한 게 뭐였지?”                                | “이전에 내가 말한 취미 기억해?”                |
| 구현 방식 | `checkpointer` (InMemorySaver, SqliteSaver 등) | `store` (InMemoryStore, DBStore 등) |
| 관리 단위 | 메시지 기록                                        | JSON 문서 (사용자 프로필, 설정, 기록 등)        |


In [1]:
from langgraph.store.memory import InMemoryStore

def embed(texts: list[str]) -> list[list[float]]:
    # 실제 임베딩 함수나 LangChain 임베딩 객체로 교체해야 합니다.
    return [[1.0, 2.0] * len(texts)]

# InMemoryStore는 메모리를 메모리(dictionary)에 저장합니다.
# 실제 서비스에서는 DB 기반 저장소를 사용하는 것이 좋습니다.
store = InMemoryStore(index={"embed": embed, "dims": 2}) 

user_id = "my-user"
application_context = "chitchat"
namespace = (user_id, application_context)  # 네임스페이스: 사용자 및 앱 컨텍스트로 구분

# 메모리 저장
store.put( 
    namespace,
    "a-memory",
    {
        "rules": [
            "사용자는 짧고 직설적인 문장을 선호합니다.",
            "사용자는 영어와 파이썬만 사용합니다.",
        ],
        "my-key": "my-value",
    },
)

# ID로 특정 메모리 조회
item = store.get(namespace, "a-memory") 

# 동일한 네임스페이스 내에서 필터 조건 및 유사도 검색 수행
items = store.search( 
    namespace, filter={"my-key": "my-value"}, query="language preferences"
)
items

[Item(namespace=['my-user', 'chitchat'], key='a-memory', value={'rules': ['사용자는 짧고 직설적인 문장을 선호합니다.', '사용자는 영어와 파이썬만 사용합니다.'], 'my-key': 'my-value'}, created_at='2025-10-27T02:25:00.821043+00:00', updated_at='2025-10-27T02:25:00.821046+00:00', score=0.9999999999999998)]

### 도구에서 장기 메모리 읽기

In [5]:
from dataclasses import dataclass
from langchain_core.runnables import RunnableConfig
from langchain.agents import create_agent
from langchain.agents import AgentState
from langchain.tools import tool, ToolRuntime
from langgraph.store.memory import InMemoryStore

@dataclass
class Context:
    user_id: str

# InMemoryStore는 메모리를 메모리(dictionary)에 저장합니다.
# 실제 서비스 환경에서는 DB 기반 저장소를 사용하는 것이 좋습니다.
store = InMemoryStore()  # [!code highlight]

# 예시 데이터 저장
store.put(  # [!code highlight]
    ("users",),  # 사용자 데이터를 모아두는 namespace
    "user_123",  # 사용자 ID (key)
    {
        "name": "John Smith",
        "language": "English",
    }  # 사용자 정보
)

@tool
def get_user_info(runtime: ToolRuntime[Context, AgentState]) -> str:
    store = runtime.store
    user_id = runtime.context.user_id
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# 에이전트 생성
agent = create_agent(
    model,
    tools=[get_user_info],
    store=store,  # [!code highlight] — 도구가 접근할 수 있도록 store를 전달
    context_schema=Context
)

# 실행 예시
agent.invoke(
    {"messages": [{"role": "user", "content": "사용자 정보를 조회해줘"}]},
    context=Context(user_id="user_123")  # [!code highlight]
)

ValueError: Function must have a docstring if description not provided.